In [1]:
import sys
import subprocess
import importlib

# ---- CONFIGURATION ----
REQUIRED_PYTHON = (3, 9)
REQUIRED_PACKAGES = [
    "pandas",
    "openpyxl",
    "sqlalchemy",
    "psycopg[binary]",
    "jupyterlab",
]

# ---- PYTHON VERSION CHECK ----
if sys.version_info < REQUIRED_PYTHON:
    raise SystemExit(
        f"❌ Python {REQUIRED_PYTHON[0]}.{REQUIRED_PYTHON[1]}+ required, "
        f"found {sys.version_info.major}.{sys.version_info.minor}"
    )
else:
    print(f"✅ Python version OK: {sys.version_info.major}.{sys.version_info.minor}")

# ---- PACKAGE CHECK / AUTO-INSTALL ----
def install(pkg):
    """Install a package via pip in the current environment."""
    print(f"⬇️  Installing missing package: {pkg}")
    subprocess.check_call([sys.executable, "-m", "pip", "install", pkg])

missing = []
for pkg in REQUIRED_PACKAGES:
    # psycopg[binary] isn't a real import name, so handle that separately
    import_name = pkg.split("[")[0]
    try:
        importlib.import_module(import_name)
    except ImportError:
        missing.append(pkg)

if missing:
    print(f"\n⚠️ Missing packages detected: {missing}")
    for pkg in missing:
        install(pkg)
else:
    print("✅ All required packages are already installed.")

# ---- SHOW INSTALLED VERSIONS ----
print("\n📦 Installed versions:")
for pkg in REQUIRED_PACKAGES:
    pkg_base = pkg.split("[")[0]
    try:
        mod = importlib.import_module(pkg_base)
        version = getattr(mod, "__version__", "unknown")
        print(f"  - {pkg_base} == {version}")
    except ImportError:
        print(f"  - {pkg_base} not installed")


✅ Python version OK: 3.10
✅ All required packages are already installed.

📦 Installed versions:
  - pandas == 2.3.3
  - openpyxl == 3.1.5
  - sqlalchemy == 2.0.44
  - psycopg == 3.2.12
  - jupyterlab == 4.4.10


In [2]:
!pip freeze > requirements.txt
print("✅ requirements.txt updated")

✅ requirements.txt updated


In [32]:
# --- CONFIG ---
from pathlib import Path

# PostgreSQL
PG_USER = "postgres"
PG_PASS = "greenage"
PG_HOST = "localhost"
# PG_HOST = "192.168.100.17"
# PG_USER = "agronomics"
# PG_PASS = "Agronomics/psql!!!"
# PG_HOST = "172.16.10.7"
PG_PORT = 5432
PG_DB   = "moin_weather"   # make sure DB is UTF8
# PG_DB   = "agronomics_weather_data"   # make sure DB is UTF8
PG_SCHEMA = "lgs2"
TABLE_NAME = "varieties_stages"

# Excel source
EXCEL_PATH = Path(r"./Variety Data V5.0 (4).xlsx")
SHEET_NAME = "Sunflower"
USECOLS    = "A:L"     # adjust if needed
HEADER_ROW = 0          # Excel row 4 -> pandas header=3 (0-indexed)

# --- ENGINE ---
from sqlalchemy import create_engine, text

conn_str = f"postgresql+psycopg://{PG_USER}:{PG_PASS}@{PG_HOST}:{PG_PORT}/{PG_DB}"
# client_encoding can be SET after connect; psycopg3 also accepts options, but SET works reliably
engine = create_engine(conn_str, pool_pre_ping=True)

with engine.connect() as conn:
    server_enc = conn.execute(text("SHOW SERVER_ENCODING;")).scalar_one()
    conn.execute(text("SET client_encoding TO 'UTF8';"))
    client_enc = conn.execute(text("SHOW CLIENT_ENCODING;")).scalar_one()
    print("SERVER_ENCODING:", server_enc)
    print("CLIENT_ENCODING:", client_enc)


SERVER_ENCODING: UTF8
CLIENT_ENCODING: UTF8


In [33]:
import pandas as pd

# Read as strings first to keep full control over conversion
df_raw = pd.read_excel(
    EXCEL_PATH,
    sheet_name=SHEET_NAME,
    header=HEADER_ROW,
    usecols=USECOLS,
    dtype=str,
    engine="openpyxl",
)

# strip whitespace in *all* string cells
df_raw = df_raw.applymap(lambda x: x.strip() if isinstance(x, str) else x)

# drop fully empty rows
df_raw = df_raw.dropna(how="all")

print("Shape after load & trim:", df_raw.shape)
print("Raw columns:")
print(df_raw.columns.tolist())

display(df_raw.head())


Shape after load & trim: (44, 12)
Raw columns:
['Crop_Name', 'Principal_Stage', 'Main_Stage', 'Sub_Stage', 'Start_GDD', 'End_GDD', 'Daily_N_Kg/ha', 'Daily_P_Kg/ha', 'Daily_K_Kg/ha', 'Crop_Coefficient', 'K_Ext (PAR)', 'SalineSensitivity']


C:\Users\ali_raza\AppData\Local\Temp\ipykernel_13744\4073561252.py:14: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_raw = df_raw.applymap(lambda x: x.strip() if isinstance(x, str) else x)


,Crop_Name,Principal_Stage,Main_Stage,Sub_Stage,Start_GDD,End_GDD,Daily_N_Kg/ha,Daily_P_Kg/ha,Daily_K_Kg/ha,Crop_Coefficient,K_Ext (PAR),SalineSensitivity
0,sunflower,Germination,Germination,00: Dry seed (achene),0,0,0.02,0.005,0.02,0.3,0.45,0.88
1,sunflower,Germination,Germination,01: Beginning of seed imbibition,1,10,NaN,NaN,NaN,NaN,NaN,NaN
2,sunflower,Germination,Germination,03: Seed imbibition complete,11,20,NaN,NaN,NaN,NaN,NaN,NaN
3,sunflower,Germination,Germination,05: Radicle emerged from seed,21,50,NaN,NaN,NaN,NaN,NaN,NaN
4,sunflower,Germination,Germination,"06: Radicle elongated, root hairs developing",51,70,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
import re

def to_snake(s: str) -> str:
    s = str(s).strip()
    s = re.sub(r"[^\w\s]+", " ", s)  # punctuation -> spaces
    s = re.sub(r"\s+", "_", s)       # whitespace -> underscore
    return s.lower()

snake_map = {c: to_snake(c) for c in df_raw.columns}

# Apply initial snake-case
df_snake = df_raw.rename(columns=snake_map)

# --- Your custom overrides (keys are snake-case) ---
# e.g. raw "Crop Name" -> snake "crop_name" -> override to "crop_fk"
custom_overrides = {
    # "crop_name": "crop_fk",
    "variety_name": "variety_fk",
    "k_ext_par_": "k_ext_par",
    # "opt__temp": "opt_temp"
    }

# Apply overrides
rename_map = {**{v: v for v in df_snake.columns}, **custom_overrides}
df_renamed = df_snake.rename(columns=rename_map)

print("Column rename preview:")
display(pd.DataFrame({
    "original": list(df_raw.columns),
    "snake": [snake_map[c] for c in df_raw.columns],
    "final": [rename_map.get(snake_map[c], snake_map[c]) for c in df_raw.columns],
}))

print("Renamed columns:")
print(df_renamed.columns.tolist())
display(df_renamed.head())


Column rename preview:


,original,snake,final
0,Crop_Name,crop_name,crop_name
1,Principal_Stage,principal_stage,principal_stage
2,Main_Stage,main_stage,main_stage
3,Sub_Stage,sub_stage,sub_stage
4,Start_GDD,start_gdd,start_gdd
5,End_GDD,end_gdd,end_gdd
6,Daily_N_Kg/ha,daily_n_kg_ha,daily_n_kg_ha
7,Daily_P_Kg/ha,daily_p_kg_ha,daily_p_kg_ha
8,Daily_K_Kg/ha,daily_k_kg_ha,daily_k_kg_ha
9,Crop_Coefficient,crop_coefficient,crop_coefficient


Renamed columns:
['crop_name', 'principal_stage', 'main_stage', 'sub_stage', 'start_gdd', 'end_gdd', 'daily_n_kg_ha', 'daily_p_kg_ha', 'daily_k_kg_ha', 'crop_coefficient', 'k_ext_par', 'salinesensitivity']


,crop_name,principal_stage,main_stage,sub_stage,start_gdd,end_gdd,daily_n_kg_ha,daily_p_kg_ha,daily_k_kg_ha,crop_coefficient,k_ext_par,salinesensitivity
0,sunflower,Germination,Germination,00: Dry seed (achene),0,0,0.02,0.005,0.02,0.3,0.45,0.88
1,sunflower,Germination,Germination,01: Beginning of seed imbibition,1,10,NaN,NaN,NaN,NaN,NaN,NaN
2,sunflower,Germination,Germination,03: Seed imbibition complete,11,20,NaN,NaN,NaN,NaN,NaN,NaN
3,sunflower,Germination,Germination,05: Radicle emerged from seed,21,50,NaN,NaN,NaN,NaN,NaN,NaN
4,sunflower,Germination,Germination,"06: Radicle elongated, root hairs developing",51,70,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
# Put your exact desired order here (subset or superset is OK)
REQUIRED_ORDER = [
    # ---- identifiers / descriptors ----
    "uuid",
    "main_stage",
    "principal_stage",
    "bbch_scale",
    "daily_n_kg_ha",
    "daily_p_kg_ha",
    "daily_k_kg_ha",
    "crop_coefficient",
    "k_ext_par",
    "saline_sensitivity",
    "variety_fk",
    # ---- batch timestamps ----
    "created_at",
    "updated_at",
]

# Ensure presence; create missing with NA
for col in REQUIRED_ORDER:
    if col not in df_renamed.columns:
        df_renamed[col] = pd.NA

# Put required columns first, keep any extras at the end (in their current order)
ordered = [c for c in REQUIRED_ORDER if c in df_renamed.columns]
extras  = [c for c in df_renamed.columns if c not in ordered]
df_ordered = df_renamed[ordered + extras].copy()

print("Final order (first 11 shown):", (ordered + extras)[:11], "...")
display(df_ordered.head())


Final order (first 11 shown): ['uuid', 'main_stage', 'principal_stage', 'bbch_scale', 'daily_n_kg_ha', 'daily_p_kg_ha', 'daily_k_kg_ha', 'crop_coefficient', 'k_ext_par', 'saline_sensitivity', 'variety_fk'] ...


,uuid,main_stage,principal_stage,bbch_scale,daily_n_kg_ha,daily_p_kg_ha,daily_k_kg_ha,crop_coefficient,k_ext_par,saline_sensitivity,variety_fk,created_at,updated_at,crop_name,sub_stage,start_gdd,end_gdd,salinesensitivity
0,<NA>,Germination,Germination,<NA>,0.02,0.005,0.02,0.3,0.45,<NA>,<NA>,<NA>,<NA>,sunflower,00: Dry seed (achene),0,0,0.88
1,<NA>,Germination,Germination,<NA>,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,sunflower,01: Beginning of seed imbibition,1,10,NaN
2,<NA>,Germination,Germination,<NA>,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,sunflower,03: Seed imbibition complete,11,20,NaN
3,<NA>,Germination,Germination,<NA>,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,sunflower,05: Radicle emerged from seed,21,50,NaN
4,<NA>,Germination,Germination,<NA>,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,sunflower,"06: Radicle elongated, root hairs developing",51,70,NaN


In [36]:
import pandas as pd
from decimal import Decimal, ROUND_HALF_UP

# ---- classify columns ----
INT_COLS = [
    "bbch_scale"
]

NUMERIC_2DP = [
    "daily_n_kg_ha",
    "daily_p_kg_ha",
    "daily_k_kg_ha",
    "crop_coefficient",
    "k_ext_par",
    "saline_sensitivity",
]

TS_COLS = [
    "created_at",
    "updated_at"
    ]

data = df_ordered.copy()

# ---- normalize strings ----
string_cols = [c for c in data.columns if c not in INT_COLS + NUMERIC_2DP + TS_COLS]
for c in string_cols:
    data[c] = data[c].astype("string").str.strip().str.lower()

# ---- integers ----
for c in INT_COLS:
    if c in data.columns:
        data[c] = pd.to_numeric(data[c], errors="coerce").astype("Int64")

# ---- helper to fix scale with Decimal ----
def to_decimal_series(s: pd.Series, places: int):
    q = Decimal("0." + "0"*places) if places > 0 else Decimal("0")
    return (pd.to_numeric(s, errors="coerce")
              .round(places)                              # numeric rounding (optional)
              .astype("object")                           # switch out of float dtype
              .map(lambda x: None if pd.isna(x) else Decimal(str(x)).quantize(q, rounding=ROUND_HALF_UP)))

# ---- 2dp & 3dp numerics as Decimal ----
for c in NUMERIC_2DP:
    if c in data.columns:
        data[c] = to_decimal_series(data[c], 2)

# ---- timestamps ----
current_ts = pd.Timestamp.now(tz="UTC")
data["created_at"] = current_ts
data["updated_at"] = current_ts

# ---- ensure NULLs ----
data = data.where(pd.notna(data), None)

print("✅ Cleaning complete. dtypes summary:")
display(data.dtypes)
display(data.head())


✅ Cleaning complete. dtypes summary:


uuid                       string[python]
main_stage                 string[python]
principal_stage            string[python]
bbch_scale                          Int64
daily_n_kg_ha                      object
daily_p_kg_ha                      object
daily_k_kg_ha                      object
crop_coefficient                   object
k_ext_par                          object
saline_sensitivity                 object
variety_fk                 string[python]
created_at            datetime64[us, UTC]
updated_at            datetime64[us, UTC]
crop_name                  string[python]
sub_stage                  string[python]
start_gdd                  string[python]
end_gdd                    string[python]
salinesensitivity          string[python]
dtype: object

,uuid,main_stage,principal_stage,bbch_scale,daily_n_kg_ha,daily_p_kg_ha,daily_k_kg_ha,crop_coefficient,k_ext_par,saline_sensitivity,variety_fk,created_at,updated_at,crop_name,sub_stage,start_gdd,end_gdd,salinesensitivity
0,<NA>,germination,germination,<NA>,0.02,0.00,0.02,0.30,0.45,None,<NA>,2026-01-13 05:23:30.846736+00:00,2026-01-13 05:23:30.846736+00:00,sunflower,00: dry seed (achene),0,0,0.88
1,<NA>,germination,germination,<NA>,None,None,None,None,None,None,<NA>,2026-01-13 05:23:30.846736+00:00,2026-01-13 05:23:30.846736+00:00,sunflower,01: beginning of seed imbibition,1,10,<NA>
2,<NA>,germination,germination,<NA>,None,None,None,None,None,None,<NA>,2026-01-13 05:23:30.846736+00:00,2026-01-13 05:23:30.846736+00:00,sunflower,03: seed imbibition complete,11,20,<NA>
3,<NA>,germination,germination,<NA>,None,None,None,None,None,None,<NA>,2026-01-13 05:23:30.846736+00:00,2026-01-13 05:23:30.846736+00:00,sunflower,05: radicle emerged from seed,21,50,<NA>
4,<NA>,germination,germination,<NA>,None,None,None,None,None,None,<NA>,2026-01-13 05:23:30.846736+00:00,2026-01-13 05:23:30.846736+00:00,sunflower,"06: radicle elongated, root hairs developing",51,70,<NA>


In [37]:
from sqlalchemy import MetaData, Table, Column, text, ForeignKey
from sqlalchemy import String, Integer, DateTime, Numeric
from sqlalchemy.dialects.postgresql import UUID

metadata = MetaData(schema=PG_SCHEMA)

table = Table(
    TABLE_NAME, metadata,
    # identifiers
    Column("uuid",                UUID(as_uuid=True), primary_key=True),
    Column("main_stage",          String),          # short labels -> String
    Column("principal_stage",     String),
    Column("bbch_scale",          Integer),
    Column("daily_n_kg_ha",       Numeric(10, 2)),
    Column("daily_p_kg_ha",       Numeric(10, 2)),
    Column("daily_k_kg_ha",       Numeric(10, 2)),
    Column("crop_coefficient",    Numeric(6, 2)),   # kc
    Column("k_ext_par",           Numeric(6, 2)),
    Column("saline_sensitivity",  Numeric(6, 2)),
    Column("variety_fk",          String, ForeignKey(f"{PG_SCHEMA}.crop_varieties.variety_eng", onupdate="CASCADE", ondelete="RESTRICT", 
                                                     name="fk_varieties_stages__variety_fk__crop_varieties_variety_eng",), nullable=False),
    Column("created_at",          DateTime(timezone=True)),
    Column("updated_at",          DateTime(timezone=True)),
    schema=PG_SCHEMA,
)

with engine.begin() as conn:
    conn.execute(text(f'CREATE SCHEMA IF NOT EXISTS "{PG_SCHEMA}";'))

    # reflect the parent table so SQLAlchemy knows about it
    crop_varieties = Table(
        "crop_varieties",
        metadata,
        schema=PG_SCHEMA,
        autoload_with=conn,      # <- reflect existing structure
    )

    metadata.create_all(conn)
    print(f"✅ Ensured table {PG_SCHEMA}.{TABLE_NAME} exists (or was created).")


✅ Ensured table lgs2.varieties_stages exists (or was created).


In [38]:
from sqlalchemy import String as SA_String, Integer as SA_Integer, DateTime as SA_DateTime, Numeric as SA_Numeric
from sqlalchemy.dialects.postgresql import UUID as SA_UUID

dtype_map = {
    "uuid":               SA_UUID(as_uuid=True),
    "main_stage":         SA_String(),
    "principal_stage":    SA_String(),
    "bbch_scale":         SA_Integer(),
    "daily_n_kg_ha":      SA_Numeric(10, 2),
    "daily_p_kg_ha":      SA_Numeric(10, 2),
    "daily_k_kg_ha":      SA_Numeric(10, 2),
    "crop_coefficient":   SA_Numeric(6, 2),
    "k_ext_par":          SA_Numeric(6, 2),
    "saline_sensitivity": SA_Numeric(6, 2),
    "variety_fk":         SA_String(),
    "created_at":         SA_DateTime(timezone=True),
    "updated_at":         SA_DateTime(timezone=True),
}


In [12]:
# from uuid import UUID, uuid4
# import pandas as pd

# # --- choose one or more varieties to upload (case-insensitive) ---
# TARGET_VARIETIES = [
#     "sugar baby",
#     "sweet beauty",
#     "golden midget",
#     "crimson sweet",
#     "king of hearts",
#     "queen of hearts",
#     "fiesta",
#     "royal sweet",
#     "starbright",
#     "anarkali",
#     "black beauty",
#     "hybrid 313",
#     "tiffany",
#     "golden crown",
# ]

# if "variety_fk" not in data.columns:
#     raise KeyError("Expected column 'variety_fk' not found in DataFrame")

# # ---- filter to target varieties (case-insensitive) ----
# var_norm = data["variety_fk"].astype("string").str.strip().str.casefold()
# targets_norm = [v.casefold() for v in TARGET_VARIETIES]
# mask = var_norm.isin(targets_norm)

# total_rows = len(data)
# kept_rows = int(mask.sum())

# if kept_rows == 0:
#     raise ValueError(
#         f"No rows matched variety_fk in {TARGET_VARIETIES}. Aborting upload."
#     )
# elif kept_rows < total_rows:
#     others = (
#         data.loc[~mask, "variety_fk"]
#         .dropna().astype("string").str.strip().str.casefold().unique().tolist()
#     )
#     print(
#         f"ℹ️ Filtering to {TARGET_VARIETIES}. "
#         f"Kept {kept_rows}/{total_rows} rows. Excluded varieties present: {others}"
#     )

# data = data.loc[mask].copy()

# # (optional) assert all remaining are in target set
# assert set(
#     data["variety_fk"].astype("string").str.strip().str.casefold().unique()
# ) <= set(targets_norm), "Found non-target variety rows after filtering."

# # ---- UUID: ensure present and valid for every row ----
# def _coerce_uuid(x):
#     if x is None or x is pd.NA or (isinstance(x, float) and pd.isna(x)) or (isinstance(x, str) and x.strip() == ""):
#         return uuid4()
#     try:
#         return x if isinstance(x, UUID) else UUID(str(x))
#     except Exception:
#         return uuid4()

# if "uuid" not in data.columns:
#     data["uuid"] = [uuid4() for _ in range(len(data))]
# else:
#     data["uuid"] = data["uuid"].map(_coerce_uuid)

# if data["uuid"].isna().any():
#     raise RuntimeError("UUID still has nulls after coercion; aborting.")

# # ---- DEDUPE: keep one principal_stage per main_stage *per variety* ----
# # normalize keys
# data["main_stage_key"] = data["main_stage"].astype("string").str.strip().str.casefold()
# data["principal_stage_key"] = data["principal_stage"].astype("string").str.strip().str.casefold()
# data["variety_key"] = data["variety_fk"].astype("string").str.strip().str.casefold()

# before = len(data)
# data = (
#     data.sort_values(["variety_key", "main_stage_key", "principal_stage_key"], kind="mergesort")
#         .drop_duplicates(subset=["variety_key", "main_stage_key", "principal_stage_key"], keep="first")
#         .reset_index(drop=True)
# )
# after = len(data)
# if after < before:
#     print(f"ℹ️ Dedupe: kept {after}/{before} rows after ensuring unique (variety, main_stage, principal_stage).")

# # ---- BBCH SCALE: map principal stages to 0..9 *per variety* ----
# BBCH_ORDER = [
#     "germination",
#     "leaf development (main shoot)",
#     "tillering",
#     "stem elongation",
#     "booting",
#     "inflorescence emergence (heading)",
#     "flowering (anthesis)",
#     "development of fruit",
#     "ripening",
#     "senescence",
# ]
# _order_map = {s: i for i, s in enumerate(BBCH_ORDER)}

# ps_key = data["principal_stage"].astype("string").str.strip().str.casefold()
# data["bbch_scale"] = ps_key.map(_order_map).astype("Int64")

# missing = data["bbch_scale"].isna().sum()
# if missing:
#     unmapped = sorted(ps_key[data["bbch_scale"].isna()].unique().tolist())
#     print(f"⚠️ {missing} row(s) have unmapped principal_stage: {unmapped}. "
#           f"Add them to BBCH_ORDER if needed.")

# # ---- cleanup helper columns ----
# data = data.drop(columns=["main_stage_key", "principal_stage_key", "variety_key"])

# # ---- drop any columns you don’t want in DB (already handled above if needed) ----
# for drop_col in [
#     "crop_name", "sub_stage", "start_gdd", "end_gdd", "start_day", "end_day",
#     "avg_daily_gdd", "stage_lai", "stage_rue", "stage_rootdepth_cm", "dm_fm",
# ]:
#     if drop_col in data.columns:
#         print(f"🧹 Dropping '{drop_col}' from DataFrame before upload")
#         data = data.drop(columns=[drop_col])

# # ---- order by variety + BBCH before upload ----
# if "bbch_scale" in data.columns:
#     data = (
#         data.sort_values(by=["variety_fk", "bbch_scale"], kind="mergesort")
#             .reset_index(drop=True)
#     )
#     print("✅ Data ordered by (variety_fk, bbch_scale) before upload.")
# else:
#     print("⚠️ bbch_scale column not found; skipping sort.")

# # ---- upload ----
# data.to_sql(
#     name=TABLE_NAME,
#     con=engine,
#     schema=PG_SCHEMA,
#     if_exists="append",
#     index=False,
#     chunksize=10_000,
#     method="multi",
#     dtype=dtype_map,
# )

# with engine.connect() as conn:
#     count = conn.execute(text(f'SELECT COUNT(*) FROM "{PG_SCHEMA}"."{TABLE_NAME}";')).scalar_one()

# print(
#     f"✅ Upload complete (varieties={TARGET_VARIETIES}). "
#     f"Row count in {PG_SCHEMA}.{TABLE_NAME}: {count}"
# )

In [39]:
# Get unique principal_stage values as a Python list
principal_stage_list = (
    data["principal_stage"]
    .dropna()
    .astype(str)
    .str.strip()
    .unique()
    .tolist()
)

print(principal_stage_list)


['germination', 'leaf development', 'stem elongation', 'inflorescence emergence', 'flowering', 'development of fruit', 'ripening', 'senescence']


In [40]:
# --------------------------------------------------
# BBCH mapping (ACTIVE – required)
# --------------------------------------------------
ps_key = (
    data["principal_stage"]
    .astype("string")
    .str.strip()
    .str.casefold()
)

BBCH_MAP = {
    "germination": 0,
    "leaf development": 1,
    "stem elongation": 3,
    "inflorescence emergence": 5,
    "flowering": 6,
    "development of fruit": 7,
    "ripening": 8,
    "senescence": 9,
}

data["bbch_scale"] = ps_key.map(BBCH_MAP).astype("Int64")

# ---- validate ----
missing = data["bbch_scale"].isna()
if missing.any():
    unmapped = sorted(ps_key[missing].unique().tolist())
    raise ValueError(
        f"❌ Unmapped principal_stage values detected:\n{unmapped}"
    )

print("✅ BBCH scale computed successfully")
display(
    data[["principal_stage", "bbch_scale"]]
    .drop_duplicates()
    .sort_values("bbch_scale")
)


✅ BBCH scale computed successfully


,principal_stage,bbch_scale
0,germination,0
8,leaf development,1
16,stem elongation,3
21,inflorescence emergence,5
26,flowering,6
31,development of fruit,7
35,ripening,8
41,senescence,9


In [41]:
# --------------------------------------------------
# Build canonical stage template (ONE row per BBCH)
# --------------------------------------------------
stage_template = (
    data
    .sort_values("bbch_scale")              # deterministic choice
    .drop_duplicates(subset=["bbch_scale"]) # 👈 KEY LINE
    .reset_index(drop=True)
)

print(f"✅ Canonical stage template rows: {len(stage_template)}")
display(stage_template[["bbch_scale", "principal_stage"]])


✅ Canonical stage template rows: 8


,bbch_scale,principal_stage
0,0,germination
1,1,leaf development
2,3,stem elongation
3,5,inflorescence emergence
4,6,flowering
5,7,development of fruit
6,8,ripening
7,9,senescence


In [42]:
from uuid import uuid4
from sqlalchemy import text
import pandas as pd

# --------------------------------------------------
# 1. Define TARGET varieties
# --------------------------------------------------
TARGET_VARIETIES = ['fh 331', 'hks 278', 'hysun 33', 'hysun 34', 'nk 265', 'aguara 4', 'pi 6480', 'sf 187', 't-40318', 'nk armani', 's-278', 'us444', 'parson3', 'oxsen 5270', 'oxsen 5264', 'hsf 350', 'orisun516', 'orisun648', 'orisun701', 'us666']

targets_norm = [v.strip().casefold() for v in TARGET_VARIETIES]

# --------------------------------------------------
# 2. Fetch varieties from DB
# --------------------------------------------------
with engine.connect() as conn:
    db_varieties = conn.execute(
        text(f'SELECT variety_eng FROM "{PG_SCHEMA}"."crop_varieties";')
    ).scalars().all()

if not db_varieties:
    raise RuntimeError("No varieties found in crop_varieties table")

db_norm_map = {v.strip().casefold(): v for v in db_varieties}

# --------------------------------------------------
# 3. Validate TARGET varieties exist in DB
# --------------------------------------------------
missing = sorted(set(targets_norm) - set(db_norm_map))
if missing:
    raise ValueError(
        "❌ These TARGET_VARIETIES are missing in crop_varieties table:\n"
        f"{missing}"
    )

print(f"✅ All {len(TARGET_VARIETIES)} target varieties exist in DB")

# --------------------------------------------------
# 4. Validate stage template
# --------------------------------------------------
if len(stage_template) != 8:
    raise RuntimeError(
        f"Stage template must have 8 rows, found {len(stage_template)}"
    )

# --------------------------------------------------
# 5. Fan-out from canonical stage template
# --------------------------------------------------
blocks = []

for v_norm in targets_norm:
    variety_eng = db_norm_map[v_norm]

    df_v = stage_template.copy()
    df_v["variety_fk"] = variety_eng
    df_v["uuid"] = [uuid4() for _ in range(len(df_v))]

    blocks.append(df_v)

data = pd.concat(blocks, ignore_index=True)

# --------------------------------------------------
# 6. Structural safety checks
# --------------------------------------------------
expected_stage_count = len(stage_template)

counts = data.groupby("variety_fk").size()
if not (counts == expected_stage_count).all():
    raise RuntimeError("Stage count mismatch between varieties")

required_cols = {
    "uuid", "main_stage", "principal_stage", "bbch_scale",
    "daily_n_kg_ha", "daily_p_kg_ha", "daily_k_kg_ha",
    "crop_coefficient", "k_ext_par", "saline_sensitivity",
    "variety_fk", "created_at", "updated_at",
}

missing_cols = required_cols - set(data.columns)
if missing_cols:
    raise RuntimeError(f"Missing required columns: {missing_cols}")

print(
    f"✅ Prepared {len(data)} rows "
    f"({len(TARGET_VARIETIES)} varieties × {expected_stage_count} stages)"
)


✅ All 20 target varieties exist in DB
✅ Prepared 160 rows (20 varieties × 8 stages)


In [45]:
# display(
#     data[["main_stage", "principal_stage", "bbch_scale"]]
#     .drop_duplicates()
#     .sort_values("bbch_scale")
# )


In [55]:
# print(len(data))
# print(data.head())

In [43]:
TABLE_COLUMNS = [
    "uuid",
    "main_stage",
    "principal_stage",
    "bbch_scale",
    "daily_n_kg_ha",
    "daily_p_kg_ha",
    "daily_k_kg_ha",
    "crop_coefficient",
    "k_ext_par",
    "saline_sensitivity",
    "variety_fk",
    "created_at",
    "updated_at",
]


In [44]:
data_db = data[TABLE_COLUMNS].copy()

In [45]:
from sqlalchemy import text

# --------------------------------------------------
# Upload to PostgreSQL (DB-safe)
# --------------------------------------------------
data_db.to_sql(
    name=TABLE_NAME,
    con=engine,
    schema=PG_SCHEMA,
    if_exists="append",
    index=False,
    chunksize=10_000,
    method="multi",
    dtype=dtype_map,
)

# --------------------------------------------------
# Verify row count
# --------------------------------------------------
with engine.connect() as conn:
    total = conn.execute(
        text(f'SELECT COUNT(*) FROM "{PG_SCHEMA}"."{TABLE_NAME}";')
    ).scalar_one()

print(f"✅ Upload complete. Total rows now in {PG_SCHEMA}.{TABLE_NAME}: {total}")


✅ Upload complete. Total rows now in lgs2.varieties_stages: 1030


In [46]:
print("🔍 Unique variety_fk values found in data:")
display(
    data["variety_fk"]
    .dropna()
    .astype("string")
    .str.strip()
    .str.casefold()
    .unique()
)


🔍 Unique variety_fk values found in data:


<StringArray>
[    'fh 331',    'hks 278',   'hysun 33',   'hysun 34',     'nk 265',
   'aguara 4',    'pi 6480',     'sf 187',    't-40318',  'nk armani',
      's-278',      'us444',    'parson3', 'oxsen 5270', 'oxsen 5264',
    'hsf 350',  'orisun516',  'orisun648',  'orisun701',      'us666']
Length: 20, dtype: string

In [47]:
# Pandas-side dtypes (should match what we intended)
print("Pandas dtypes:")
display(data.dtypes)

# DB-side types (optional: inspect)
qry = f"""
SELECT column_name, data_type, is_nullable
FROM information_schema.columns
WHERE table_schema = :schema AND table_name = :table
ORDER BY ordinal_position;
"""
with engine.connect() as conn:
    rows = conn.execute(text(qry), {"schema": PG_SCHEMA, "table": TABLE_NAME}).mappings().all()
pd.DataFrame(rows)


Pandas dtypes:


uuid                               object
main_stage                 string[python]
principal_stage            string[python]
bbch_scale                          Int64
daily_n_kg_ha                      object
daily_p_kg_ha                      object
daily_k_kg_ha                      object
crop_coefficient                   object
k_ext_par                          object
saline_sensitivity                 object
variety_fk                         object
created_at            datetime64[us, UTC]
updated_at            datetime64[us, UTC]
crop_name                  string[python]
sub_stage                  string[python]
start_gdd                  string[python]
end_gdd                    string[python]
salinesensitivity          string[python]
dtype: object

,column_name,data_type,is_nullable
0,uuid,uuid,NO
1,main_stage,character varying,YES
2,principal_stage,character varying,YES
3,bbch_scale,integer,YES
4,daily_n_kg_ha,numeric,YES
5,daily_p_kg_ha,numeric,YES
6,daily_k_kg_ha,numeric,YES
7,crop_coefficient,numeric,YES
8,k_ext_par,numeric,YES
9,saline_sensitivity,numeric,YES
